# Evolutionary Algorithms

Evolutionary algorithms are a family of population-based and typically gradient-free optimization methods for non-linear / non-convex functions in high dimensions. Certain variants are well suited for multi-objective optimization as population converges to sampling the Pareto front.

* [Differential evolution](#Differential-evolution)
* [Particle swarm optimization](#Particle-swarm-optimization)
* [Genetic algorithms](#Genetic-algorithms)
* [CMA-ES](#CMA-ES)

In [1]:
import numpy as np

## Genetic algorithms

The idea is to consider the fixed-size vector encoding of an individual as its genes. Individuals are selected and paired according to their fitness, and offsprings are created by a crossover operation of the parents genes with some mutations.

## Differential evolution

The idea is to loop over individuals and
* (mutation) randomly pick 3 other individuals $a, b, c$ from population to construct new traits as $x_n = a + (b - c) \times F$ where $F$ is the mutation rate
* (crossover) randomly accept new traits into the individual according to crossover probability

In [2]:
def minimize_de(function, bounds, steps=50, n=20, mutation=0.5, crossover=0.7):
    """Differential evolution optimization
    
    Args:
        function: callable f(x) to be minimized
        bounds: box bounds of the input space [[lower bounds], [upper bounds]]
        steps: number of optimization steps
        n: population size, >= 4
        mutation: mutation rate / differential weight, typically (0 - 2]
        crossover: crossover probability
        
    Returns:
        x_min: the best inputs found
    """
    lower, upper = bounds
    d = len(lower)  # number of inputs
    
    population = np.random.uniform(lower, upper, size=(n, d))
            
    for i in range(steps):
        
        # loop over individuals in the population
        for j, x_target in enumerate(population):
            
            # mutation
            candidates = list(range(n))
            candidates.remove(j)
            x1, x2, x3 = population[np.random.choice(candidates, size=3, replace=False)]
            x_new = x1 + (x2 - x3) * mutation
            x_new = np.clip(x_new, lower, upper)

            # cross-over
            x_new = np.where(np.random.rand(d) < crossover, x_new, x_target)
                    
            # selection
            if function(x_new) < function(x_target):
                population[j] = x_new

    best = np.argmin(function(population))
    return population[best]


def some_function(x):
    return np.sum((x - [1, 0, 0])**2, axis=-1)

minimize_de(some_function, bounds=[[-2, -2, -2], [2, 2, 2]])

array([9.99931034e-01, 3.38251861e-05, 2.24834883e-05])

## Particle swarm optimization

The idea is that individuals (called particles here) start walking in a random direction but are pulled towards their own best known optimum and the best known optimum of the swarm.

Basic implemtation according https://en.wikipedia.org/wiki/Particle_swarm_optimization#Algorithm

In [3]:
def minimize_pso(function, bounds, steps=50, n=20, ω=0.5, φ1=0.2, φ2=0.2):
    """Particle swarm optimization
    
    Args:
        function: callable f(x) to be minimized
        bounds: box bounds of the input space [[lower bounds], [upper bounds]]
        steps: number of optimization steps
        n: population size
        ω: inertia
        φ1: pull towards particle's best knwown position
        φ2: pull towards swarm's best known position

    Returns:
        x_min: the best inputs found
    """
    lower, upper = bounds
    d = len(lower)  # number of inputs
    
    # initial position and velocity
    X = np.random.uniform(lower, upper, size=(n, d))
    V = np.random.uniform(-1, 1, size=(n, d)) * (upper - lower)

    # best known position
    X_best = X.copy()
    
    # swarm's best known position
    x_swarm = X[np.argmin(function(X))]
    
    # while termination criterion not met
    for _ in range(steps):
        
        # loop over particles
        for i in range(n):
            
            # update velocity
            r1 = np.random.uniform(0, φ1, size=d)
            r2 = np.random.uniform(0, φ2, size=d)
            V[i] = ω * V[i] + r1 * (X_best[i] - X[i]) + r2 * (x_swarm - X[i])
            
            # update position
            X[i] = np.clip(X[i] + V[i], lower, upper)
            
            # update the particles best known position
            if function(X[i]) < function(X_best[i]):
                X_best[i] = X[i]
            
            # update the swarms best known position
            if function(X_best[i]) < function(x_swarm):
                x_swarm = X_best[i]

    return x_swarm


def some_function(x):
    return np.sum((x - [1, 0, 0])**2, axis=-1)

bounds = np.array([[-2, -2, -2], [2, 2, 2]])
minimize_pso(some_function, bounds)

array([ 1.00122578e+00, -5.14263650e-03,  3.90900545e-04])

## CMA-ES

[Covariance matrix adaption evolution strategy](https://en.wikipedia.org/wiki/CMA-ES) [tutorial](https://arxiv.org/abs/1604.00772)

In an evolution strategy, new candidate solutions are sampled according to a multivariate normal distribution. 
Recombination amounts to selecting a new mean value for the distribution.
Mutation amounts to adding a random perturbation. 
Pairwise dependencies between the variables in the distribution are represented by a covariance matrix.
The covariance matrix adaptation (CMA) is a method to update the covariance matrix of this distribution

In the commonly used (μ/μw, λ)-CMA-ES in each iteration step a weighted combination of the μ best out of λ new candidate solutions is used to update the distribution parameters.